In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import wandb

In [18]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [19]:
# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="aml_lab1_1",

    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.01,
    "architecture": "CNN",
    "dataset": "FashionMNIST",
    "epochs": 50,
    }
)

epoch,▁▅█
training_accuracy,▁▇█
training_loss,█▂▁
validation_loss,█▁▂
epoch,3
training_accuracy,89.33333
training_loss,0.29001
validation_loss,0.33084


In [20]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.fc1(x.view(-1, 28 * 28))
        x = nn.functional.relu(x)
        x = self.fc2(x)
        x = nn.functional.relu(x)
        x = self.fc3(x)
        return x

class CnnNet(nn.Module):
    def __init__(self):
        super(CnnNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = self.pool2(x)
        x = x.view(-1, 64 * 7 * 7)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)
        return x

In [21]:
transform = transforms.ToTensor()
train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

In [ ]:
def GetModelAndResults(train_dataset, test_dataset, type="linear", epochs=50):
  train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
  test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
  if type == "linear":
    model = SimpleNet()
  elif type == "cnn":
    model = CnnNet()
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=0.01)

  for epoch in range(epochs):
    # Training phase
    model.train()
    training_loss = 0
    total_correct = 0
    total_samples = 0
    
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        training_loss += loss.item()
        
        # Compute accuracy (optional)
        _, predicted = outputs.max(1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)
    
    training_loss /= len(train_loader)  # Average training loss
    training_accuracy = total_correct / total_samples * 100

    # Validation phase
    model.eval()
    validation_loss = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            validation_loss += loss.item()
    
    validation_loss /= len(test_loader)  # Average validation loss

    # Log metrics to wandb
    wandb.log({
        "epoch": epoch + 1,
        "training_loss": training_loss,
        "validation_loss": validation_loss,
        "training_accuracy": training_accuracy
    })

    print(f"Epoch {epoch+1}: Training Loss: {training_loss:.4f}, Validation Loss: {validation_loss:.4f}, Training Accuracy: {training_accuracy:.2f}%")

  # Evaluation
  model.eval()
  correct = 0
  total = 0
  with torch.no_grad():
      for images, labels in test_loader:
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  accuracy = 100 * correct / total
  print(f'Accuracy: {accuracy}%')
  # Save your script file as an artifact
  artifact = wandb.Artifact("training_script", type="code")
  artifact.add_file("AML_Lab1.py")
  wandb.log_artifact(artifact)
  wandb.finish()
  return model, accuracy

In [23]:
# wandb.init(
#     project="aml_lab1_1",
#     config={
#     "learning_rate": 0.01,
#     "architecture": "linear",
#     "dataset": "FashionMNIST",
#     "epochs": 50,
#     }
# )
# linearModel, linearAccuracy = GetModelAndResults(train_dataset, test_dataset, type="linear", epochs=50)

# wandb.init(
#     project="aml_lab1_1",
#     config={
#     "learning_rate": 0.01,
#     "architecture": "CNN",
#     "dataset": "FashionMNIST",
#     "epochs": 50,
#     }
# )
cnnModel, cnnAccuracy = GetModelAndResults(train_dataset, test_dataset, type="cnn", epochs=20)

Epoch 1: Training Loss: 0.4638, Validation Loss: 0.3865, Training Accuracy: 82.86%
Epoch 2: Training Loss: 0.3376, Validation Loss: 0.3685, Training Accuracy: 87.32%
Epoch 3: Training Loss: 0.3138, Validation Loss: 0.3478, Training Accuracy: 88.35%
Epoch 4: Training Loss: 0.2964, Validation Loss: 0.3415, Training Accuracy: 88.87%
Epoch 5: Training Loss: 0.2870, Validation Loss: 0.3259, Training Accuracy: 89.13%
Epoch 6: Training Loss: 0.2796, Validation Loss: 0.3347, Training Accuracy: 89.44%
Epoch 7: Training Loss: 0.2702, Validation Loss: 0.3255, Training Accuracy: 89.91%
Epoch 8: Training Loss: 0.2655, Validation Loss: 0.3231, Training Accuracy: 90.04%
Epoch 9: Training Loss: 0.2611, Validation Loss: 0.3431, Training Accuracy: 90.24%
Epoch 10: Training Loss: 0.2576, Validation Loss: 0.3335, Training Accuracy: 90.42%
Epoch 11: Training Loss: 0.2517, Validation Loss: 0.3494, Training Accuracy: 90.58%
Epoch 12: Training Loss: 0.2478, Validation Loss: 0.3317, Training Accuracy: 90.83%
E

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
training_accuracy,▁▄▅▆▆▆▆▇▇▇▇▇▇▇█▇████
training_loss,█▄▄▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁
validation_loss,▇▅▃▃▁▂▁▁▃▂▃▂▄▂▃▂▃█▄▇
epoch,20
training_accuracy,91.88167
training_loss,0.21823
validation_loss,0.39393
